# Timeline Visualisation

This notebook creates JSON files of exhibition data and renders them as a timeline visualisation using the Timeline code from KnightLab https://timeline.knightlab.com/ 

# Import libraries and variables 

In [34]:
import json
import ipywidgets
from IPython.display import display, HTML,clear_output


%store -r activity_dir
%store -r activity_all_file
%store -r datavis_events_all_timeline
%store -r datavis_venue_exhibitions
%store -r datavis_venue_timeline
%store -r json_suffix


## Create JSON data files from Linked Art for exhibitions

### Create a timeline JSON file for all exhibition events.

In [35]:
# Make JSON data file


events_list = []

with open(os.path.join(activity_dir, activity_all_file), 'r') as json_file:
    data = json.load(json_file)
    for event in data.get("events"):

        start = event.get("start").split("T")[0]

        start_year = start.split("-")[0]
        start_month = start.split("-")[1]
        start_day = start.split("-")[2]

        end = event.get("start").split("T")[0]

        end_year = start.split("-")[0]
        end_month = start.split("-")[1]
        end_day = start.split("-")[2]
         
        

        display_date = str(start) + " - " + str(end)

        headline = event.get("_label")

        event_vis = {
      "media": {
        "url": "",
        "caption": "",
        "credit": "",
        "thumbnail": ""
      },
      "display_date": display_date,
      "start_date": {
        "year": start_year,
        "month": start_month,
        "day": start_day
      },
      "end_date": {
        "year": end_year,
        "month": end_month,
        "day": end_day
      },
      "text": {
        "headline": headline,
        "text": ""
      },
      "type": "title",
      "background": {  "color": "#3f83e8"  }
    }
        events_list.append(event_vis)

with open(datavis_events_all_timeline, 'w') as file:
        file.write(json.dumps({"events": events_list},indent=2))

### Create individual JSON files of exhibitions for each venue

In [36]:

with open(os.path.join(datavis_venue_exhibitions, 'index.json'), 'r') as json_file:
    data = json.load(json_file)

    for venue in data.get("venues"):
      events_list = []

      filename_venue = venue.get("filename") + ".json"
      with open(os.path.join(datavis_venue_exhibitions, filename_venue), 'r') as venue_file:
        data = json.load(venue_file)
     

        for event in data.get("events"):

          start = event.get("start").split("T")[0]
          start_year = start.split("-")[0]
          start_month = start.split("-")[1]
          start_day = start.split("-")[2]

          end = event.get("start").split("T")[0]
          end_year = start.split("-")[0]
          end_month = start.split("-")[1]
          end_day = start.split("-")[2]
          display_date = str(start) + " - " + str(end)
          headline = event.get("_label")

          event_vis = {
              "media": {   "url": "",   "caption": "",   "credit": "",  "thumbnail": ""   },
              "display_date": display_date,
              "start_date": {   "year": start_year,  "month": start_month, "day": start_day },
              "end_date": { "year": end_year, "month": end_month, "day": end_day},
              "text": {   "headline": headline,"text": ""  },
              "type": "title",
              "background": {  "color": "#3f83e8"  }
            }
          events_list.append(event_vis)

      
      with open(os.path.join(datavis_venue_timeline, filename_venue) , 'w') as file:
        file.write(json.dumps({"events": events_list},indent=2))

## Timeline visualisation

Example visualisation for the Hansa Gallery

In [37]:
local_html_file = '<head>\
</head>\
<body>\
        <link title="timeline-styles" rel="stylesheet"  href="https://cdn.knightlab.com/libs/timeline3/latest/css/timeline.css"/>\
        <script src="https://cdn.knightlab.com/libs/timeline3/latest/js/timeline.js"></script>\
            <h1>Timeline of exhibition data</h1>\
        <div id="timeline-embed" style="width: 100%; height: 700px"></div>\
        <script type="text/javascript"> window.timeline = new TL.Timeline("timeline-embed", "'
local_html_file +=  datavis_venue_timeline
local_html_file += '/HansaGallery.json");</script>\
</body>'

HTML(local_html_file)

## Data Visualisation

Timeline data visualisations for all venues. Select venue from the drop-down and the corresponding timeline will be displayed.

In [38]:



selectOptions = []
selectOptions = [('Please select a venue', '')]



with open(os.path.join(datavis_venue_exhibitions, 'index.json'), 'r') as json_file:
    data = json.load(json_file)
    for venue in data.get("venues"):
        filename = venue.get("filename")
        name = venue.get("name")
        selectOptions.append( (name , filename))



def dropdown_eventhandler(change):
    clear_output()
    local_html_file = ""
    filename_venue=change.new
    with open(os.path.join(datavis_venue_timeline, filename_venue + '.json'), 'r') as venue_file:
        
        local_html_file = '<link title="timeline-styles" rel="stylesheet"  href="https://cdn.knightlab.com/libs/timeline3/latest/css/timeline.css"/><script src="https://cdn.knightlab.com/libs/timeline3/latest/js/timeline.js"></script>'

        local_html_file += '<h1>Timeline of exhibition data ' + filename_venue + '</h1>\
        <div id="timeline-embed-' + venue.get("filename") + '" style="background-color:blue;width: 100%; height: 700px">' + filename_venue +'</div>\
        <script type="text/javascript"> window.timeline = new TL.Timeline("timeline-embed-' + venue.get("filename") + '", "' 
        local_html_file += datavis_venue_timeline + filename_venue + json_suffix
        local_html_file +=  '");</script>'
    
    selectObject = ipywidgets.Dropdown(options=selectOptions)
    selectObject.observe(dropdown_eventhandler, names='value')
    display(selectObject) 
    
    display(HTML(local_html_file))
    

selectObject = ipywidgets.Dropdown(options=selectOptions)
selectObject.observe(dropdown_eventhandler, names='value')
display(selectObject)   
    


Dropdown(options=(('Please select a venue', ''), ('42nd Venice Biennale', '42ndVeniceBiennale'), ('44th Venice…